# Guided Playbook Runner — BEC: Individual Email Compromise (IEC)

This notebook is a **repo add-on** that helps you run the IEC playbook consistently and generate an **incident report skeleton**.

## How to use
1. Edit the **Incident Context** cell.
2. Run cells top-to-bottom.
3. Find generated files under `../outputs/`.

> Tip: keep timestamps in **UTC** in the report. Convert user-reported local times and record both when needed.

In [ ]:
from dataclasses import dataclass, asdict
from datetime import datetime, timezone
from pathlib import Path
import json

OUTPUT_DIR = Path("../outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

@dataclass
class IncidentContext:
    incident_id: str
    victim_email: str
    severity: str              # SEV1/2/3/4
    status: str                # Open/Contained/Closed
    detected_time_utc: str     # ISO 8601 string
    contained_time_utc: str    # ISO 8601 string or ""
    summary: str               # 1–3 sentences
    suspected_vector: str      # phishing/oauth/token/stuffing/unknown
    time_window_start_utc: str # ISO 8601 string
    time_window_end_utc: str   # ISO 8601 string

# ---- EDIT THESE ----
ctx = IncidentContext(
    incident_id="IEC-2026-0001",
    victim_email="victim@example.com",
    severity="SEV2",
    status="Open",
    detected_time_utc="2026-02-10T09:30:00Z",
    contained_time_utc="",
    summary="Suspicious sign-in followed by inbox rule creation; outbound payment request emails observed.",
    suspected_vector="phishing",
    time_window_start_utc="2026-02-09T00:00:00Z",
    time_window_end_utc="2026-02-10T23:59:59Z",
)

print(json.dumps(asdict(ctx), indent=2))

## Operational Checklist (first actions)

- **Evidence:** export sign-in + audit logs; preserve sample emails with full headers.
- **Containment:** disable sign-in → revoke sessions → reset password → reset MFA → remove forwarding/rules/delegates → revoke OAuth grants.
- **Business risk:** alert Finance/AP if there’s *any* payment-change or invoice fraud pattern.

In [ ]:
from datetime import datetime

def build_task_list(ctx: IncidentContext):
    tasks = [
        ("Identify", "Confirm compromise via sign-in/audit evidence"),
        ("Identify", "Establish time window (last known good → containment)"),
        ("Identify", f"Determine initial vector (current guess: {ctx.suspected_vector})"),
        ("Contain", "Disable sign-in / lock account"),
        ("Contain", "Revoke sessions / refresh tokens (global sign-out)"),
        ("Contain", "Reset password (force re-auth)"),
        ("Contain", "Reset MFA / require re-registration if suspicious"),
        ("Contain", "Remove inbox rules, forwarding, delegates"),
        ("Contain", "Revoke suspicious OAuth grants / block app if needed"),
        ("Contain", "Message trace outbound emails; purge/quarantine malicious messages"),
        ("Eradicate", "Disable legacy auth protocols if not required"),
        ("Eradicate", "Apply Conditional Access / device compliance controls"),
        ("Recover", "Notify internal/external recipients if phishing was sent"),
        ("Recover", "Monitor for 7–14 days (rules, forwarding, sign-ins, outbound volume)"),
        ("Learn", "Post-incident review and hardening actions"),
    ]
    return tasks

tasks = build_task_list(ctx)
for phase, item in tasks:
    print(f"[{phase}] {item}")

## Investigation Queries (copy/paste starters)

See `../queries/` for vendor-specific examples:
- `m365_kql.md`
- `google_workspace_queries.md`

In [ ]:
from string import Template

TEMPLATE_PATH = Path("../templates/incident_report.md")

def render_incident_report(ctx: IncidentContext,
                           timeline_rows="",
                           outbound_count="TBD",
                           external_recipients="TBD",
                           internal_recipients="TBD",
                           data_access="TBD",
                           financial_impact="TBD",
                           regulatory_impact="TBD",
                           ips="TBD",
                           user_agents="TBD",
                           forwarding="TBD",
                           oauth_apps="TBD",
                           domains_urls="TBD",
                           containment_actions="- TBD",
                           eradication_actions="- TBD",
                           recovery_actions="- TBD",
                           root_cause="TBD",
                           lessons_learned="- TBD"):
    raw = TEMPLATE_PATH.read_text(encoding="utf-8")
    # Use simple placeholder replacement {{key}} → value
    mapping = {
        "incident_id": ctx.incident_id,
        "victim_email": ctx.victim_email,
        "severity": ctx.severity,
        "status": ctx.status,
        "detected_time": ctx.detected_time_utc,
        "contained_time": ctx.contained_time_utc or "TBD",
        "summary": ctx.summary,
        "timeline_rows": timeline_rows or "| TBD | TBD | TBD |",
        "outbound_count": outbound_count,
        "external_recipients": external_recipients,
        "internal_recipients": internal_recipients,
        "data_access": data_access,
        "financial_impact": financial_impact,
        "regulatory_impact": regulatory_impact,
        "ips": ips,
        "user_agents": user_agents,
        "forwarding": forwarding,
        "oauth_apps": oauth_apps,
        "domains_urls": domains_urls,
        "containment_actions": containment_actions,
        "eradication_actions": eradication_actions,
        "recovery_actions": recovery_actions,
        "root_cause": root_cause,
        "lessons_learned": lessons_learned,
    }
    out = raw
    for k, v in mapping.items():
        out = out.replace("{{" + k + "}}", str(v))
    return out

report_md = render_incident_report(ctx)
out_path = OUTPUT_DIR / f"{ctx.incident_id}_incident_report.md"
out_path.write_text(report_md, encoding="utf-8")
print(f"Wrote: {out_path.resolve()}")

## Optional: Analyze exported logs (starter helpers)

If you export sign-in logs or mailbox rules to CSV, you can drop them into `../outputs/` and adapt the helpers below.
These are intentionally lightweight and won’t cover every field in every tenant.

In [ ]:
import pandas as pd

def load_csv(path):
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"Not found: {p.resolve()}")
    return pd.read_csv(p)

def quick_signin_triage(df, user_col_candidates=("UserPrincipalName","userPrincipalName","User","user")):
    # Try to detect the user column
    user_col = next((c for c in user_col_candidates if c in df.columns), None)
    if user_col is None:
        user_col = df.columns[0]

    victim = ctx.victim_email.lower()
    dfx = df.copy()
    dfx[user_col] = dfx[user_col].astype(str).str.lower()
    dfx = dfx[dfx[user_col].str.contains(victim, na=False)]

    # Common useful columns (if present)
    cols = [c for c in ["TimeGenerated","CreatedDateTime","timestamp","IPAddress","ipAddress","Location","location","ClientAppUsed","clientAppUsed","DeviceDetail","deviceDetail","ResultType","resultType","Status","status"] if c in dfx.columns]
    if cols:
        dfx = dfx[cols]
    return dfx.sort_values(by=cols[0] if cols else dfx.columns[0], ascending=False)

# Example usage:
# df = load_csv("../outputs/signins.csv")
# triaged = quick_signin_triage(df)
# triaged.head(20)

## Close-out checklist (don’t skip)

- Confirm: no forwarding, no malicious rules, no unknown delegates.
- Confirm: OAuth grants reviewed; sessions revoked; MFA is re-registered.
- Notify: impacted recipients (internal/external) if necessary.
- Monitor: 7–14 days; add alerts for new rules and risky sign-ins.
- Retrospective: document root cause and control improvements.